# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [20]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [21]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [22]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,On the determination of shear strength from te...,Conference proceedings presentation,ECCS21,21st European Conference on Composie Materials...,07/02/2024,"Nantes, France",https://eccm21.org,"Authors: Maria Gfrerrer, Vasco D.C. Pires, A.J..."
1,3D printing solutions to fight COVID-19,Conference poster,EUGLOH,EUGLOH Annual Student Research Conference,28/09/2020,NaN,https://www.eugloh.eu/events/eugloh-annual-stu...,"Authors: Vasco D.C Pires, Cristina Ferreira, J..."
2,3D-Printing to Fight COVID-19,Conference proceedings presentation,engBrasil2020,engBRASIL2020 - lst Brazilian Engineering Cong...,03/11/2020,NaN,https://www.redepdimat.org/engbrasil2020,"Authors: Vasco D.C Pires, Ana Pais, Cristina F..."
3,Desenvolvimento de Dispositivos Impressos em 3...,Conference proceedings presentation,SPB9,9º Congresso Nacional De Biomecânica,19/02/2021,"Porto, Portugal",https://cnbio2021.wixsite.com/cnb2021,"Authors: Ana Pais, Cristina Ferreira, Vasco D...."
4,M5 - Mars Magnetospheric Multipoint Measuremen...,Talk,M5AG2022,Annual Meeting of the German Astronomical Soci...,12/09/2022,"Bremen, Germany",https://ag2022.astronomische-gesellschaft.de/v...,"Authors: M5 Team - Cormac Larkin, Leonard Schu..."
5,M5 - A multi-spacecraft plasma physics misson ...,Conference poster,M5Finland,Winter Satellite Workshop 2024,17/01/2024,"Alto, Finland",https://spaceworkshop.fi,"Authors: M5 Team - Vile Ludén, Cormac Larkin, ..."
6,Enhancing Joint Performance with Residual Stre...,Conference poster,IJUP2023,IJUP 2023 – Youth Research of the University o...,10/05/2023,"Porto, Portugal",https://www.up.pt/ijup/ijup-2023/,"Authors: Vasco D.C Pires, Ricardo J.C Carbas, ..."
7,3D printed devices for avoiding hand contact w...,Conference proceedings presentation,SPB10,10° Congresso Nacional De Biomecânica,05/05/2023,"Figueira da Foz, Portugal",https://10cnb2023.pt/,"Authors: Ana Pais, Cristina Ferreira, Vasco D...."
8,MVSE Mission Phase A/0 Study: A Proposal for U...,Conference poster,EGU23,"EGU23, the 25th EGU General Assembly,",23/04/2023,"Vienna, Austria",https://meetingorganizer.copernicus.org/EGU23/...,"Authors: Nadim Maraqten, Viktoria Kutnohorsky,..."
9,Optimization of dissimilar single-lap joints b...,Conference poster,Ab2023P1,AB2023 - 7th International Conference On Adhes...,13/07/2023,"Porto, Portugal",https://fe.up.pt/abconference/2023/,"Authors: Vasco D.C Pires, Ricardo J.C Carbas, ..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [23]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [24]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    #if len(str(item.talk_url)) > 3:
    #    md += "\n[Conference/Talk Website](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [25]:
!ls ../_talks

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
!cat ../_talks/2013-03-01-tutorial-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
